# Nyquist Sampling and Reconstruction Demo

This notebook mirrors `samplig_reconstruction.py` and visualizes how sampling at or below the Nyquist rate affects reconstruction quality. We generate a band-limited signal, sample it at Nyquist and undersampled rates, and then rebuild the signal via sinc interpolation to quantify mean-squared error and highlight phase alignment effects.


In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
np.random.seed(0)



In [ ]:
def generate_signal(t: np.ndarray) -> np.ndarray:
    freq1, freq2 = 1.0, 3.0
    return np.sin(2 * np.pi * freq1 * t) + 0.5 * np.sin(2 * np.pi * freq2 * t)


def reconstruct_signal(t_dense, t_samples, samples, T):
    x = (t_dense[:, None] - t_samples[None, :]) / T
    return np.sum(samples[None, :] * np.sinc(x), axis=1)


t = np.linspace(0, 1, 1000)
original = generate_signal(t)

f_max = 5
fs_nyquist = 2 * f_max
T_nyquist = 1 / fs_nyquist
fs_under = 1.5 * f_max
T_under = 1 / fs_under

samples_idx_ny = np.arange(0, 1 + T_nyquist, T_nyquist)
samples_idx_under = np.arange(0, 1 + T_under, T_under)

samples_ny = generate_signal(samples_idx_ny)
samples_under = generate_signal(samples_idx_under)

recon_ny = reconstruct_signal(t, samples_idx_ny, samples_ny, T_nyquist)
recon_under = reconstruct_signal(t, samples_idx_under, samples_under, T_under)

mse_ny = np.mean((original - recon_ny) ** 2)
mse_under = np.mean((original - recon_under) ** 2)
print(f"MSE (Nyquist): {mse_ny:.6f}")
print(f"MSE (Undersampled): {mse_under:.6f}")



In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
axes[0].plot(t, original, label="Original", color="black")
axes[0].plot(t, recon_ny, label="Nyquist reconstruction", linestyle="--", color="#1b9e77")
axes[0].set_title("Nyquist-sampled signal")
axes[0].legend()

axes[1].plot(t, original, label="Original", color="black")
axes[1].plot(t, recon_under, label="Undersampled reconstruction", linestyle="--", color="#d95f02")
axes[1].set_title("Undersampled signal")
axes[1].legend()
axes[1].set_xlabel("Time (s)")

plt.tight_layout()
plt.show()



Nyquist sampling (top) perfectly reconstructs the band-limited signal, while undersampling introduces aliasing error and visibly distorted phase. The same gap motivates the semantic Nyquist threshold for prompts: too few contextual samples prevent stable reconstruction of the latent concept.
